<a href="https://colab.research.google.com/github/wenqinglim/hum_me_a_melody/blob/main/hum_me_a_melody_gradio_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Could you please hum me a melody? 🥺 🎤  



In [ ]:
# These 2 packages are for avoiding this bug in transformers on colab: https://stackoverflow.com/questions/76902752/importerror-using-low-cpu-mem-usage-true-or-a-device-map-requires-accelerat
# Note: They require restarting the session
!pip install --upgrade --quiet pip
!pip install --quiet accelerate
!pip install -U --quiet peft

# Force-restart session
exit()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Install ffmpeg for processing audio and video
!pip install --quiet ffmpeg-python

# Essentia for tagging the music

!pip install --quiet essentia-tensorflow
!wget https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.pb
!wget https://essentia.upf.edu/models/classification-heads/mtg_jamendo_moodtheme/mtg_jamendo_moodtheme-discogs-effnet-1.pb


# Install kerykeion, to get today's astrology information
!pip install --quiet kerykeion

# Install audiocraft, which requires torch 2.1.0
!pip install --quiet torch==2.1.0+cu121 torchvision torchtext==0.16.0 torchaudio==2.1.0+cu121 -f https://download.pytorch.org/whl/torch_stable.html
!pip install --quiet git+https://github.com/facebookresearch/audiocraft.git


# Install AniimateDiff-Lightning (video generation)
!pip install --quiet diffusers[torch]

# Gradio for UI
!pip install --quiet gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.4/291.4 MB 5.0 MB/s eta 0:00:00
--2024-04-26 21:01:14--  https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18366619 (18M) [application/octet-stream]
Saving to: ‘discogs-effnet-bs64-1.pb’

discogs-effnet-bs64 100%[===================>]  17.52M  1.30MB/s    in 15s     

2024-04-26 21:01:29 (1.15 MB/s) - ‘discogs-effnet-bs64-1.pb’ saved [18366619/18366619]

--2024-04-26 21:01:30--  https://essentia.upf.edu/models/classification-heads/mtg_jamendo_moodtheme/mtg_jamendo_moodtheme-discogs-effnet-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 273966

In [ ]:
# A bug in Colab causes this error: https://github.com/googlecolab/colabtools/issues/3409
# These 2 lines ensure that the bug does not occur during this notebook.
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import numpy as np
import ffmpeg

In [ ]:
# Mood detection on audio
#@title Load the MTG Listening Models

# Thank you Dr Colton for the MTG model code snippets from lab notebooks.

from essentia.standard import AudioLoader, MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D
import essentia

embeddings_model = TensorflowPredictEffnetDiscogs(
    graphFilename="discogs-effnet-bs64-1.pb",
    output="PartitionedCall:1",
)

mood_classification_model = TensorflowPredict2D(
    graphFilename="mtg_jamendo_moodtheme-discogs-effnet-1.pb",
    output='model/Sigmoid',
)

# Functions for Using the Listening Models

all_tags = [
  "action", "adventure", "advertising",
  "background", "ballad",
  "calm", "children", "christmas", "commercial", "cool", "corporate",
  "dark", "deep", "documentary", "drama", "dramatic", "dream",
  "emotional", "energetic", "epic",
  "fast", "film", "fun", "funny",
  "game", "groovy",
  "happy", "heavy", "holiday", "hopeful",
  "inspiring",
  "love",
  "meditative", "melancholic", "melodic", "motivational", "movie",
  "nature",
  "party", "positive", "powerful",
  "relaxing", "retro", "romantic",
  "sad", "sexy", "slow", "soft", "soundscape", "space", "sport", "summer",
  "trailer", "travel",
  "upbeat", "uplifting"
]

# Tags categorised with help from ChatGPT (checked and refined by me)
# Moods
mood_tags = [
    "calm", "cool",
    "dark", "deep", "dramatic",
    "emotional", "energetic", "epic",
    "fast", "fun", "funny",
    "groovy",
    "happy", "heavy", "hopeful",
    "inspiring",
    "meditative", "melancholic", "motivational",
    "positive", "powerful",
    "relaxing", "romantic", "sad", "sexy", "slow", "soft",
     "upbeat", "uplifting"
]

# Themes
theme_tags = [
    "action", "adventure", "ballad",
    "children", "christmas",
     "dream", "film", "game",
    "holiday", "love", "movie",
    "nature", "party", "retro", "space","sport",
    "summer", "travel"
]

# Functions
function_tags = [
    "advertising", "background", "commercial",
    "corporate", "documentary", "drama",
    "soundscape", "trailer"
]

def get_mood_activations_dict(audio):
  embeddings = embeddings_model(audio)

  activations = mood_classification_model(embeddings)
  activation_avs = []
  for i in range(0, len(activations[0])):
    vals = [activations[j][i] for j in range(0, len(activations))]
    # Note - this does the averaging bit
    activation_avs.append(sum(vals)/len(vals))
  activations_dict = {}
  for ind, tag in enumerate(all_tags):
    activations_dict[tag] = activation_avs[ind]

  return activations_dict


def get_moods(audio):

  # Get mood activations
  mood_activations_dict = get_mood_activations_dict(essentia.array(audio))

  # Calculate upper threshold for outliers
  q1 = np.quantile(list(mood_activations_dict.values()), 0.25)
  q3 = np.quantile(list(mood_activations_dict.values()), 0.75)
  # Outliers defined as Q3 + 1.5*IQR
  outlier_threshold = q3 + (1.5*(q3-q1))

  # Select moods that are above outlier threshold (i.e. most prominent moods)
  tags = [tag for tag, score in mood_activations_dict.items() if (score >= outlier_threshold) & (tag != 'melodic')]
  moods = [tag for tag in tags if tag in mood_tags]
  themes = [tag for tag in tags if tag in theme_tags]
  functions = [tag for tag in tags if tag in function_tags]

  return moods, themes, functions

In [ ]:
# @title functions for Astrology information and instrumentation
from kerykeion import AstrologicalSubject
from datetime import datetime

import random

# Define Holst's planets instrumentation and sound descriptions
# Definition generated by ChatGPT (checked by me)
planets_instruments = {
    'Mars': {
        'instruments': ['trombone', 'trumpet'],
        'sound': 'Aggressive, martial rhythms and powerful brass fanfares'
    },
    'Venus': {
        'instruments': ['french horn'],
        'sound': 'Warm, mellow tones symbolizing serenity and tranquility'
    },
    'Mercury': {
        'instruments': ['flute', 'clarinet'],
        'sound': 'Light and nimble melodies evoking swift movements'
    },
    'Jupiter': {
        'instruments': ['french horn', 'violin'],
        'sound': 'Majestic brass fanfares and lush string melodies'
    },
    'Saturn': {
        'instruments': ['organ'],
        'sound': 'Deep, resonant tones adding a sense of gravity and solemnity'
    },
    'Uranus': {
        'instruments': ['timpani', 'brass'],
        'sound': 'Bold rhythms and unconventional brass and percussion usage'
    },
    'Neptune': {
        'instruments': ['celesta', 'harp'],
        'sound': 'Ethereal, shimmering tones creating an otherworldly atmosphere'
    }
}

holst_planets = list(planets_instruments.keys())

# Helper function to get today's planets, based on an Astrological sign
# Note: We default the Astrological sign to Aries, since this idea was conceived in April
# Alternatively, the sign could be set to the audience's sign, but our AI system wants its own sign too

def get_planets(filtered_sign='Ari', valid_planets=holst_planets):
  """
  Returns list of planets that are in the filtered_sign today.

  """
  today = datetime.now()

  # Create object for astrological subject
  astro_subj = AstrologicalSubject("GenAI", today.year, today.month, today.day, today.hour, today.minute, "London", "GB")


  today_planets = []
  for planet in astro_subj.planets_list:
    if (planet['sign'] == filtered_sign) & (planet['name'] in valid_planets):
      today_planets.append(planet['name'])

  if len(today_planets) == 0:
    # If no planets are in filtered_sign today, pick a random planet
    today_planets.append(random.choice(holst_planets))

  return today_planets

In [ ]:
# @title Setup MusicGen

# Import music generation model
import torch
import torchaudio

device = "cuda:0" if torch.cuda.is_available() else "cpu"

from audiocraft.models import MusicGen
from audiocraft.utils.notebook import display_audio
from audiocraft.data.audio import audio_write

def construct_musicgen_prompt(moods, themes, functions, planets, planets_instruments):
  moods_str = ", ".join(moods)

  prompt = f"a {moods_str} orchestral piece"
  if themes:
    prompt += f" with {', '.join(themes)} themes"
  for planet in planets:
    planet_instr = planets_instruments[planet]['instruments']
    planet_sound = planets_instruments[planet]['sound']
    planet_str = f"{', '.join(planet_instr)} with {str.lower(planet_sound)}"
    prompt += f". {planet_str}"
  if functions:
    prompt += f"perfect for {', '.join(functions)}"
  return prompt


def generate_music(musicgen_prompt, audio, sr):
  """
  Generates music using pre-trained MusicGen model
  """
  model = MusicGen.get_pretrained('melody')
  model.set_generation_params(duration=8)  # generate 8 seconds.

  # generates using the melody from the given audio and the provided descriptions.
  wav_gpu = model.generate_with_chroma([musicgen_prompt], torch.tensor(audio).float().reshape(1, 1, -1), sr)[0]
  wav = wav_gpu.cpu()
  del wav_gpu

  # Will save under composition.wav, with loudness normalization at -14 db LUFS.
  audio_write(f'composition', wav, model.sample_rate, strategy="loudness")

  return wav



/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_hip.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
# @title Setup AniimateDiff-Lightning (video generation)

# Setup video generation pipeline
# Note: This takes up ~4GB of GPU RAM
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_video

from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import moviepy.editor


dtype = torch.float16
step = 4  # Options: [1,2,4,8]
repo = "ByteDance/AnimateDiff-Lightning"
ckpt = f"animatediff_lightning_{step}step_diffusers.safetensors"
base = "emilianJR/epiCRealism"

# Load Motion Low-Rank Adaptations (LoRAs), which determines the type of motion in the videos
adapter = MotionAdapter().to(device, dtype)
adapter.load_state_dict(load_file(hf_hub_download(repo ,ckpt), device=device))

# Setup pipeline with base model and animateDiff-lightning motion adaptor
pipe = AnimateDiffPipeline.from_pretrained(base, motion_adapter=adapter, torch_dtype=dtype).to(device)

# Set motion type
pipe.load_lora_weights(
    "guoyww/animatediff-motion-lora-zoom-out", adapter_name="zoom-out"
)

# Setup scheduler
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing", beta_schedule="linear")

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



(…)ff_lightning_4step_diffusers.safetensors:   0%|          | 0.00/908M [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  warnings.warn(

The config attributes {'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'mid_block_type': 'UNetMidBlock2DCrossAttn', 'only_cross_attention': False, 'attention_head_dim': 8, 'dual_cross_attention': False, 'class_embed_type': None, 'addition_embed_type': None, 'num_class_embeds': None, 'upcast_attention': False, 'resnet_time_scale_shift': 'default', 'resnet_skip_time_act': False, 'resnet_out_scale_factor': 1.0, 'time_embedding_type': 'positional', 'time_embedding_dim': None, 'time_embedding_act_fn': None, 'timestep_post_act': None, 'conv_in_kernel': 3, 'conv_out_kernel': 3, 'projection_class_embeddings_input_dim': None, 'class_embeddings_concat': False, 'mid_block_only_cross_attention': None, 'cross_attention_norm': None, 'addition_embed_type_num_heads': 64} were passed to UNetMotionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


diffusion_pytorch_model.safetensors:   0%|          | 0.00/77.4M [00:00<?, ?B/s]

In [ ]:
# @title Define wrapper functions for Gradio
def melody_to_composition(audio_file):
  """
  Wrapper function for Gradio

  Takes input audio and outputs generated music with an output message detailing its creative process.
  """
  sr, audio = audio_file
  moods, themes, functions = get_moods(audio)
  message = f"I'd describe your voice and melody as {', '.join(moods)}.\
    If that's not what you intended to sound like, then maybe practice singing more. "
  if themes:
    message += f"It has some {', '.join(themes)} vibes. "
  if functions:
    message += f"And it also sounds like it can be used as {', '.join(functions)} music. "

  planets = get_planets()
  if planets:
    message += f"\nThese are the planet(s) in my astrology sign today: {planets}.\
    I have composed a piece inspired by them. "

  musicgen_prompt = construct_musicgen_prompt(moods, themes, functions, planets, planets_instruments)

  wav = generate_music(musicgen_prompt, audio, sr)

  message += f"\nBased on your melody, I'm have created {musicgen_prompt}. Take a listen - what does it remind you of? "

  torch.cuda.empty_cache()


  return "composition.wav", moods, message


def text_to_muvi(user_input_txt, moods):
  """
  Wrapper function for Gradio

  Takes user input text and generates a video.
  Outputs a combined video with audio, and a message detailing its creative process.
  """
  video_prompt = f"{user_input_txt}, {', '.join(moods)} mood"
  # Generate video (+ 10.6GB of GPU RAM usage)
  output = pipe(
                prompt=video_prompt,
                negative_prompt="bad quality, worse quality, watermark",
                guidance_scale=1.0,
                num_frames=16,
                num_inference_steps=step
                )
  export_to_video(output.frames[0]*5, "video_only.mp4")
  torch.cuda.empty_cache()

  # Combine audio and video
  audio = ffmpeg.input("composition.wav")
  video = ffmpeg.input("video_only.mp4")


  music_video = ffmpeg.concat(video, audio, v=1, a=1)

  music_video.output("muvi.mp4").run(overwrite_output=True)

  message = f"Since the music sounds like '{user_input_txt}' to you,\
  I've created a video that captures that imagery in a {', '.join(moods)} mood.\
  Here's the masterpiece: A music video!"

  return "muvi.mp4", message

In [ ]:
# @title Gradio application
import gradio as gr

demo = gr.Blocks()

with demo:

    # 1. Get users to record a melody
    gr.Markdown(
    """
    # Hello World!
    Could you please hum me a melody?
    """)
    moods = gr.State([])
    audio_file = gr.Audio(type="numpy")
    b1 = gr.Button("Submit melody")
    message = gr.Textbox()

    # 2. Let users listen to generated music
    wav_path = gr.Audio(interactive=False)
    user_text = gr.Textbox(placeholder="What does the melody remind you of?")
    b2 = gr.Button("Submit interpretation")

    # 3. Show final music video
    message_final = gr.Textbox()
    muvi_path = gr.PlayableVideo()

    b1.click(melody_to_composition, inputs=audio_file, outputs=[wav_path, moods, message])
    b2.click(text_to_muvi, inputs=[user_text, moods], outputs=[muvi_path, message_final])

    demo.unload(lambda: print("unloading"))

demo.launch(debug=True, height=1500)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://421b58129607745b2e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


  warnings.warn(



state_dict.bin:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 88.9MB/s]


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")



  0%|          | 0/4 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
unloading
Killing tunnel 127.0.0.1:7860 <> https://421b58129607745b2e.gradio.live


<video src="muvi.mp4">

<video src="muvi.mp4">